# PyTerrier Starter: Full-Rank with ChatNoir for Task 3 in Touche 2023

This is the [PyTerrier](https://github.com/terrier-org/pyterrier) baseline for [task 3 on Image Retrieval for Arguments](https://touche.webis.de/clef23/touche23-web/image-retrieval-for-arguments.html) in [Touché 2023](https://touche.webis.de/clef23/touche23-web/).

This notebook implements a re-ranking baseline that re-ranks the results of some first-stage ranker with BM25.

### Adapt the notebook locally

You can adapt/run this baseline locally with docker and can directly deploy and run it in [TIRA.io](https://www.tira.io/task/touche-task-3).

With docker installed, you can start this notebook with the command:

```
docker run --rm -ti \
    -p 8888:8888 \
    -v ${PWD}:/workspace \
    webis/tira-touche23-task-3-pyterrier-baseline:0.0.1 \
    jupyter-lab --allow-root --ip 0.0.0.0
```

### Deployment in TIRA

To deploy approaches in TIRA, you upload the image and specify the command that is to be executed in the image. TIRA gives you a personalized documentation on how to upload the image, and to run this notebook in TIRA you can specify the following command in TIRA:

```
/workspace/run-notebook.py --notebook /workspace/re-rank-pipeline.ipynb --input $inputDataset --output $outputDir
```

You can dry-run this on your machine by executing the command:

```
./run-notebook.py \
    --input ${PWD}/sample-input/re-rank \
    --output ${PWD}/sample-output \
    --notebook /workspace/re-rank-pipeline.ipynb \
    --local-dry-run True
```



### Additional Resources

- The [PyTerrier tutorial](https://github.com/terrier-org/ecir2021tutorial)
- The [PyTerrier documentation](https://pyterrier.readthedocs.io/en/latest/)
- The [TIRA quickstart](https://touche.webis.de/clef23/touche23-web/evidence-retrieval-for-causal-questions.html#tira-quickstart)



### Step 1: Import everything and load variables

In [1]:
from tira_utils import get_input_directory_and_output_directory, normalize_run
import pyterrier as pt
import pandas as pd
import os
import json
from tqdm import tqdm
from glob import glob
from pathlib import Path

SYSTEM_NAME = os.environ.get('TIRA_SYSTEM_NAME' ,'my-retrieval-system')

if not pt.started():
    # tira_utils above should already have done started pyterrier with this configuration to ensure that no internet connection is required (for reproducibility)
    pt.init(version=os.environ['PYTERRIER_VERSION'], helper_version=os.environ['PYTERRIER_HELPER_VERSION'], no_download=True)

input_directory, output_directory = get_input_directory_and_output_directory(default_input='/workspace/sample-input/re-rank')


Start PyTerrier with version=5.7, helper_version=0.0.7, no_download=True


PyTerrier 0.9.1 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


I will use a small hardcoded example located in /workspace/sample-input/re-rank.
The output directory is /tmp/


### Step 2: Load the data

In [3]:
print(f'Step 2: Read input data from {input_directory}.')

# We use some very baseline method to get a textual representation: we just use the text of the pages that contain the image.
def load_image_text(image_id):
    ret = ''
    
    for txt_file in glob(input_directory +'/images/' + image_id[:3] + '/' + image_id + '/*/*/*/text.txt'):
        ret += '\n\n' + open(txt_file).read()
        
    return ret.strip()

def load_df():
    file_name = input_directory + '/rerank.jsonl'
    
    if not os.path.exists(file_name):
        raise ValueError(f'Could not find the file "{file_name}". Got: {glob(input_directory + "/*")}')
    
    df = pd.read_json(file_name, lines=True)
    df['query'] = df['query'].apply(lambda i: "".join([x if x.isalnum() else " " for x in i]))
    df['qid'] = df['qid'].astype('str')
    df['text'] = df['docno'].apply(lambda i: load_image_text(i))
    
    return df
    
df = load_df()
print(f'Done...')

df.head(3)

Step 2: Read input data from /workspace/sample-input/re-rank.
Done...


,qid,query,docno,text
0,34,Are social networking sites good for our society,Iad17912610912ffd,"Instagram\nTwitter\nFacebook\n\nThursday, Octo..."
1,34,Are social networking sites good for our society,Ia20c1e2e90f832cb,Upload Log in\n Search\n \nVideo\nSlideshow\n...
2,34,Are social networking sites good for our society,I374eede3492beb08,Free Trial Login\n\tSearch\nFeatures\n\t\nPSHE...


### Step 3: Define the actual retrieval appraoch

In [4]:
print('Step 3: Define the retrieval approach')
bm25_scorer = pt.text.scorer(body_attr="text", wmodel='BM25', verbose=True)


Step 3: Define the retrieval approach


### Step 4: Run the pipeline

In [6]:
print('Step 4: Run the retrieval pipeline')

run = bm25_scorer(df)
run.head(3)

Step 4: Run the retrieval pipeline


51documents [00:00, 87.78documents/s]                                                                                                             
BR(BM25):   0%|                                                                                                              | 0/2 [00:00<?, ?q/s]

15:52:56.586 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


BR(BM25): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 130.17q/s]


,qid,docno,text,rank,score,query
0,34,Iad17912610912ffd,"Instagram\nTwitter\nFacebook\n\nThursday, Octo...",0,0.000000,Are social networking sites good for our society
1,34,Ia20c1e2e90f832cb,Upload Log in\n Search\n \nVideo\nSlideshow\n...,2,-0.168664,Are social networking sites good for our society
2,34,I374eede3492beb08,Free Trial Login\n\tSearch\nFeatures\n\t\nPSHE...,3,-0.851783,Are social networking sites good for our society


### Step 5: Stence Detection

In [7]:
print('Step 5: Define stence detection')

def detect_stance(query_document_pair):
    # As baseline, we return always pro
    return 'PRO'

run['Q0'] = run.apply(lambda i: detect_stance(i), axis=1)

Step 5: Define stence detection


### Step 6: Persist results

In [9]:
print('Step 6: Persist Run.')

Path(output_directory).mkdir(parents=True, exist_ok=True)
normalize_run(run, SYSTEM_NAME).to_csv(output_directory + '/run.txt', sep=' ', header=False, index=False)

print('Done...')

Step 6: Persist Run.
Done...
